<h1 align=center>MIT Covid-19 Datathon</h1>
<h2 align=center>Track D - Team 6</h2>
<h3 align=center>Data File Consolidation and Cleaning</h3>

In [1]:
# Standard Tool Imports

# We'll use the census geocoding for finding county information from lat, lon
import censusgeocode as cg

# Use pandas for dataframe tools
import pandas as pd

In [2]:
# These definitions are run time environment specific
data_path = '/home/michael/Documents/Data/safegraph/v1/main-file/'

In [3]:
# Define our operating parameters
focus_states = ['CA', 'NY', 'LA']

# Define NCIS Codes to save

# Pattern file columns to keep
pattern_keep_columns = ['safegraph_place_id', 'location_name', 'region', \
                        'postal_code', 'naics_code', 'raw_visit_counts', 'median_dwell', 'bucketed_dwell_times']

core_poi_keep_columns = ['safegraph_place_id', 'latitude', 'longitude']


In [4]:
# Define the files we want to process
visit_data_files = ['2020-04-26-weekly-patterns.csv']

core_poi_data_files = ['core_poi-part1.csv', 'core_poi-part2.csv','core_poi-part3.csv',\
                      'core_poi-part4.csv', 'core_poi-part5.csv']

In [5]:
# Read the first datafile into a dataframe
df = pd.read_csv(data_path+visit_data_files[0])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3648456 entries, 0 to 3648455
Data columns (total 23 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   safegraph_place_id         object 
 1   location_name              object 
 2   street_address             object 
 3   city                       object 
 4   region                     object 
 5   postal_code                int64  
 6   brands                     object 
 7   naics_code                 float64
 8   date_range_start           object 
 9   date_range_end             object 
 10  raw_visit_counts           int64  
 11  raw_visitor_counts         int64  
 12  visits_by_day              object 
 13  visits_by_each_hour        object 
 14  visitor_home_cbgs          object 
 15  visitor_country_of_origin  object 
 16  distance_from_home         float64
 17  median_dwell               float64
 18  bucketed_dwell_times       object 
 19  related_same_day_brand     object 
 20  re

In [7]:
# reduce the dataframe to the columns we need
df = df[pattern_keep_columns]

In [8]:
df.head()

,safegraph_place_id,location_name,region,postal_code,naics_code,raw_visit_counts,median_dwell,bucketed_dwell_times
0,sg:0009298396584acaae5fe847f36b8329,Prima Dance & Formal Boutique,SC,29485,448310.0,28,18.5,"{""<5"":1,""5-20"":14,""21-60"":10,""61-240"":3,"">240"":0}"
1,sg:000a09446a024498a59f97096f3d1b60,First Baptist Church,KS,67342,813110.0,3,40.0,"{""<5"":0,""5-20"":1,""21-60"":2,""61-240"":0,"">240"":0}"
2,sg:000bb2d76161429c93dcc7efa1613334,Mazmart,GA,30341,441310.0,21,42.0,"{""<5"":0,""5-20"":8,""21-60"":3,""61-240"":7,"">240"":3}"
3,sg:00128c68c0a44501b7ff7f5ac4fef61a,LifeLong Dental Care,CA,94703,621210.0,1,40.0,"{""<5"":0,""5-20"":0,""21-60"":1,""61-240"":0,"">240"":0}"
4,sg:00207437c9f34eaa91b06505fa5f07d8,Custom Training Group,FL,32822,624410.0,32,118.0,"{""<5"":0,""5-20"":2,""21-60"":4,""61-240"":19,"">240"":7}"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3648456 entries, 0 to 3648455
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   safegraph_place_id    object 
 1   location_name         object 
 2   region                object 
 3   postal_code           int64  
 4   naics_code            float64
 5   raw_visit_counts      int64  
 6   median_dwell          float64
 7   bucketed_dwell_times  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 222.7+ MB


In [10]:
# This function will split the bucketed dwell field and calculate an estimated mean
def calc_dwells(dwell_col) :
    total_dwell_list = []
    dwell_buckets = {}
    bucket_multipliers = {'<5' : 2.5, '5-20' : 12.5, '21-60' : 40.5, '61-240' : 150.5, '>240' : 240}
    
    # run through each row in the series
    
    for row in dwell_col :
        # ignore the starting and ending brackets
        entries = row[1:-1].split(',')

        # for each bucket, split the key from the value
        for entry in entries:
            bucket, count = entry.split(':')
            dwell_buckets[bucket.replace('\"', '')] = int(count)


            total_dwell = 0
            
            # Now use the dictionary to calculate the total dwell time
            for bucket in dwell_buckets :
                total_dwell += dwell_buckets[bucket] * bucket_multipliers[bucket]

        total_dwell_list.append(total_dwell)
    
    # return the list of total dwell times
    return(total_dwell_list)

In [11]:
df['est_total_dwell'] = calc_dwells(df['bucketed_dwell_times'])

-----

Build a dataframe of the core poi files

----

In [12]:
# This series of steps creates a dataframe with all the core poi entries

# initialize the data frame
big_core_df = pd.read_csv(data_path+core_poi_data_files[0])

# only keep the columns we'll use in our analysis
big_core_df = big_core_df[core_poi_keep_columns]

# Run through all the other files in the list and append
for file_name in core_poi_data_files[1:]:
    try:
        # read the next data file and concat on the end
        df2 = pd.read_csv(data_path+file_name)
        df2 = df2[core_poi_keep_columns]
        big_core_df = pd.concat([big_core_df, df2])
        
        # Force the deletion of df2 to save memory
        del df2
    except OSError :
        print("Unable to open file {}".format(data_path+file_name))
        continue

In [13]:
# Merge the travel pattern information with the poi info
big_df = df.merge(big_core_df, how='left')

In [14]:
# Read the datafile that contains the lat and lon of each poi
# This file was created by using the shape file filter in process.py
loc_df = pd.read_csv('placeCountyCBG.csv')

In [15]:
# Merge the location information into our main data frame
big_df = big_df.merge(loc_df, how='left')

In [16]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3648456 entries, 0 to 3648455
Data columns (total 16 columns):
 #   Column                Dtype  
---  ------                -----  
 0   safegraph_place_id    object 
 1   location_name         object 
 2   region                object 
 3   postal_code           int64  
 4   naics_code            float64
 5   raw_visit_counts      int64  
 6   median_dwell          float64
 7   bucketed_dwell_times  object 
 8   est_total_dwell       float64
 9   latitude              float64
 10  longitude             float64
 11  state                 object 
 12  stateFIPS             float64
 13  countyFIPS            float64
 14  countyName            object 
 15  CBGFIPS               float64
dtypes: float64(8), int64(2), object(6)
memory usage: 473.2+ MB


In [17]:
big_df.head()

,safegraph_place_id,location_name,region,postal_code,naics_code,raw_visit_counts,median_dwell,bucketed_dwell_times,est_total_dwell,latitude,longitude,state,stateFIPS,countyFIPS,countyName,CBGFIPS
0,sg:0009298396584acaae5fe847f36b8329,Prima Dance & Formal Boutique,SC,29485,448310.0,28,18.5,"{""<5"":1,""5-20"":14,""21-60"":10,""61-240"":3,"">240"":0}",1034.0,32.998132,-80.181953,SC,45.0,45035.0,Dorchester County,4.503501e+11
1,sg:000a09446a024498a59f97096f3d1b60,First Baptist Church,KS,67342,813110.0,3,40.0,"{""<5"":0,""5-20"":1,""21-60"":2,""61-240"":0,"">240"":0}",93.5,37.059240,-95.360933,KS,20.0,20099.0,Labette County,2.009995e+11
2,sg:000bb2d76161429c93dcc7efa1613334,Mazmart,GA,30341,441310.0,21,42.0,"{""<5"":0,""5-20"":8,""21-60"":3,""61-240"":7,"">240"":3}",1995.0,33.881720,-84.316148,GA,13.0,13089.0,DeKalb County,1.308902e+11
3,sg:00128c68c0a44501b7ff7f5ac4fef61a,LifeLong Dental Care,CA,94703,621210.0,1,40.0,"{""<5"":0,""5-20"":0,""21-60"":1,""61-240"":0,"">240"":0}",40.5,37.848760,-122.269888,CA,6.0,6001.0,Alameda County,6.001424e+10
4,sg:00207437c9f34eaa91b06505fa5f07d8,Custom Training Group,FL,32822,624410.0,32,118.0,"{""<5"":0,""5-20"":2,""21-60"":4,""61-240"":19,"">240"":7}",4726.5,28.461945,-81.297527,FL,12.0,12095.0,Orange County,1.209501e+11


In [18]:
big_df.to_csv('safegraph_intermediate.csv')

In [19]:
# Filter out just one state
new_big_df = big_df[big_df['region'] == 'CT']

In [20]:
new_big_df.head()

,safegraph_place_id,location_name,region,postal_code,naics_code,raw_visit_counts,median_dwell,bucketed_dwell_times,est_total_dwell,latitude,longitude,state,stateFIPS,countyFIPS,countyName,CBGFIPS
110,sg:03cf51aca5424a50add1aa9f026cbc06,Mobil,CT,6810,447110.0,22,10.5,"{""<5"":1,""5-20"":13,""21-60"":4,""61-240"":2,"">240"":2}",1108.0,41.399047,-73.433857,CT,9.0,9001.0,Fairfield County,9.001210e+10
119,sg:042412fca26944bebde7b45b661a89c0,Barney Library,CT,6032,519120.0,6,120.5,"{""<5"":0,""5-20"":2,""21-60"":0,""61-240"":4,"">240"":0}",627.0,41.720860,-72.829212,CT,9.0,9003.0,Hartford County,9.003460e+10
169,sg:053d8cb61f784841b6af19b26b3c224c,Pops Exhaust,CT,6042,811111.0,3,32.0,"{""<5"":0,""5-20"":1,""21-60"":1,""61-240"":1,"">240"":0}",203.5,41.804612,-72.533725,CT,9.0,9003.0,Hartford County,9.003514e+10
193,sg:063ae966c79145789c2bcdb8948b0b3e,Hartford HealthCare Medical Group,CT,6790,621111.0,5,59.0,"{""<5"":0,""5-20"":1,""21-60"":2,""61-240"":2,"">240"":0}",394.5,41.792353,-73.133997,CT,9.0,9005.0,Litchfield County,9.005311e+10
248,sg:07b288bc9c154054bdf7f42d50c4f115,Living Faith Christian Church,CT,6702,813110.0,2,28.5,"{""<5"":0,""5-20"":0,""21-60"":2,""61-240"":0,"">240"":0}",81.0,41.558701,-73.041470,CT,9.0,9009.0,New Haven County,9.009350e+10


In [21]:
new_big_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39800 entries, 110 to 3648430
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   safegraph_place_id    39800 non-null  object 
 1   location_name         39800 non-null  object 
 2   region                39800 non-null  object 
 3   postal_code           39800 non-null  int64  
 4   naics_code            39703 non-null  float64
 5   raw_visit_counts      39800 non-null  int64  
 6   median_dwell          39800 non-null  float64
 7   bucketed_dwell_times  39800 non-null  object 
 8   est_total_dwell       39800 non-null  float64
 9   latitude              39800 non-null  float64
 10  longitude             39800 non-null  float64
 11  state                 39234 non-null  object 
 12  stateFIPS             39234 non-null  float64
 13  countyFIPS            39234 non-null  float64
 14  countyName            39234 non-null  object 
 15  CBGFIPS        

In [22]:
new_big_df.reindex()

,safegraph_place_id,location_name,region,postal_code,naics_code,raw_visit_counts,median_dwell,bucketed_dwell_times,est_total_dwell,latitude,longitude,state,stateFIPS,countyFIPS,countyName,CBGFIPS
110,sg:03cf51aca5424a50add1aa9f026cbc06,Mobil,CT,6810,447110.0,22,10.5,"{""<5"":1,""5-20"":13,""21-60"":4,""61-240"":2,"">240"":2}",1108.0,41.399047,-73.433857,CT,9.0,9001.0,Fairfield County,9.001210e+10
119,sg:042412fca26944bebde7b45b661a89c0,Barney Library,CT,6032,519120.0,6,120.5,"{""<5"":0,""5-20"":2,""21-60"":0,""61-240"":4,"">240"":0}",627.0,41.720860,-72.829212,CT,9.0,9003.0,Hartford County,9.003460e+10
169,sg:053d8cb61f784841b6af19b26b3c224c,Pops Exhaust,CT,6042,811111.0,3,32.0,"{""<5"":0,""5-20"":1,""21-60"":1,""61-240"":1,"">240"":0}",203.5,41.804612,-72.533725,CT,9.0,9003.0,Hartford County,9.003514e+10
193,sg:063ae966c79145789c2bcdb8948b0b3e,Hartford HealthCare Medical Group,CT,6790,621111.0,5,59.0,"{""<5"":0,""5-20"":1,""21-60"":2,""61-240"":2,"">240"":0}",394.5,41.792353,-73.133997,CT,9.0,9005.0,Litchfield County,9.005311e+10
248,sg:07b288bc9c154054bdf7f42d50c4f115,Living Faith Christian Church,CT,6702,813110.0,2,28.5,"{""<5"":0,""5-20"":0,""21-60"":2,""61-240"":0,"">240"":0}",81.0,41.558701,-73.041470,CT,9.0,9009.0,New Haven County,9.009350e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648126,sg:f25dbe6692654011bfebf79656e668f4,Sunoco,CT,6820,447110.0,16,17.0,"{""<5"":3,""5-20"":5,""21-60"":6,""61-240"":1,"">240"":1}",703.5,41.045766,-73.482617,CT,9.0,9001.0,Fairfield County,9.001030e+10
3648234,sg:f72901cc5fb041c88653d1e28dd0a5b7,Irving Oil,CT,6073,447110.0,34,9.5,"{""<5"":2,""5-20"":21,""21-60"":5,""61-240"":2,"">240"":4}",1731.0,41.666200,-72.602534,CT,9.0,9003.0,Hartford County,9.003521e+10
3648382,sg:fd549a806f4947feb92aa2ccad7bb371,Clc William Pitt,CT,6902,624410.0,8,18.5,"{""<5"":0,""5-20"":4,""21-60"":1,""61-240"":1,"">240"":2}",721.0,41.063335,-73.527333,CT,9.0,9001.0,Fairfield County,9.001022e+10
3648409,sg:fe51b8d3c012493a8d58a67c3309ea7f,Inner Wisdom Chiropractic,CT,6340,621310.0,1,48.0,"{""<5"":0,""5-20"":0,""21-60"":1,""61-240"":0,"">240"":0}",40.5,41.346301,-72.031123,CT,9.0,9011.0,New London County,9.011703e+10


In [23]:
new_big_df.to_csv('weekly-patterns-4-26-2020-CT.csv')